# Load and Merge Data

In [3]:
import pandas as pd

The news analytics data is secured from RavenPack Analytics. The source of the news articles include Dow Jones Financial Wires, Wall Street Journal, Barron’s, MarketWatch, Press releases, regulatory, corporate and news services.

For each record in the RavenPack Analytics dataset, a set of analytics are produced:

Entities: Company details like full name, country of domicile, RavenPack’s unique entity identifier, and securities identifiers, among others.

Events: Information about the type of event detected in the news following RavenPack´s detailed event taxonomy.

Scores: A set of numerical scores identifying different aspects of an event in relation to the entity in the news (e.g. Relevance Score, Event Relevance Score, Event Similarity Days, and Event Sentiment Score).

In order to align the RP Analytics data with the daily stock data, I employ the 'Entities' attributes. This allows me to pinpoint the RP records that are directly applicable to the target company. Subsequently, I compute the daily average of the sentiment scores derived from all relevant records.

The 'Sentiment Score' column values span from -1 to 1. A score near -1 signifies that the overall news sentiment leans towards negativity, while a value closer to one indicates a predominantly positive sentiment.

For instance, consider the news sentiment score for Microsoft on 30th April 2013, marked as 0.01375. This score represents the average sentiment derived from all news articles gathered by RP Analytics concerning Microsoft on that specific day. An average sentiment score of 0.01375 suggests an almost neutral sentiment for the given day in the news pertaining to Microsoft.

In [4]:
# Load the news_data.csv file
news_data = pd.read_csv('news_data.csv')

# Load the stock_data.csv file
stock_data = pd.read_csv('stock_data.csv')

# Merge the two datasets based on "Date" and "Ticker"
merged_data = pd.merge(stock_data, news_data, on=["Date", "Ticker"])


In [5]:
news_data.describe()

,Sent
count,202738.000000
mean,0.753430
std,2.511692
min,-96.880000
25%,0.000000
50%,0.100000
75%,0.480000
max,66.620000


In [3]:
df_covid =describe.read_csv("owid-covid-data.csv")

In [4]:
# Filter the data for iso_code 'USA'
df_covid = df_covid[df_covid['iso_code'] == 'USA']

df_covid = df_covid[df_covid["date"] <= "2023-04-28"]

# Extract the day of the week for each date in the dataset
df_covid['day_of_week'] = pd.to_datetime(df_covid['date']).dt.day_name()

# Drop rows where day_of_week is either 'Saturday' or 'Sunday'
df_covid = df_covid[~df_covid['day_of_week'].isin(['Saturday', 'Sunday'])]

# Drop all columns in df_covid except for 'date' and 'new_cases'
df_covid = df_covid[['date', 'new_cases']]

# Rename the 'new_cases' column to 'new_covid_cases'
df_covid = df_covid.rename(columns={"new_cases": "New_Covid_Cases"})

# Rename the 'Sent' column to 'Sentiment_Score'
merged_data.rename(columns={"Sent": "Sentiment_Score"}, inplace=True)


In [5]:
merged_data = pd.merge(merged_data, df_covid, left_on="Date", right_on="date", how="left")

In [6]:
merged_data.drop(columns=["date"], inplace=True)

In [7]:
import yfinance as yf

# List of tickers
tickers = merged_data["Ticker"].unique().tolist()

# Fetch the market cap data
market_caps = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    try:
        market_caps[ticker] = stock.info['marketCap']
    except:
        market_caps[ticker] = None

print(market_caps)

{'MSFT': 2396829515776, 'AAPL': 2818976514048, 'NVDA': 1073202724864, 'AMZN': 1430710255616, 'META': 779736252416, 'TSLA': 754921635840, 'GOOGL': 1662264016896, 'GOOG': 1664647692288, 'AVGO': 350897799168, 'PEP': 253345775616, 'COST': 249013714944, 'ADBE': 237813645312, 'CSCO': 219340111872, 'NFLX': 189552279552, 'AMD': 178307121152, 'CMCSA': 194059026432, 'TMUS': 167156678656, 'TXN': 154187481088, 'INTC': 147610796032, 'HON': 126276526080, 'INTU': 141441499136, 'QCOM': 127280070656, 'ISRG': 107026243584, 'AMGN': 139758567424, 'AMAT': 118614261760, 'SBUX': 115293446144, 'BKNG': 119494189056, 'ADI': 92802441216, 'MDLZ': 100130021376, 'GILD': 99457458176, 'ADP': 104061550592, 'VRTX': 89939730432, 'LRCX': 89286393856, 'PYPL': 70189309952, 'REGN': 88501960704, 'PANW': 67025068032, 'ATVI': 71536541696, 'MU': 74507780096, 'CSX': 61764870144, 'SNPS': 65882238976, 'KLAC': 66925674496, 'ASML': 264468348928, 'CDNS': 61761564672, 'FTNT': 46589603840, 'ORLY': 57603616768, 'MNST': 61672153088, 'MAR

In [8]:
# Convert the market_caps dictionary to a DataFrame
df_market_caps = pd.DataFrame(list(market_caps.items()), columns=["Ticker", "MarketCap"])

# Merge the market_caps_df with merged_data using "Ticker" as key and "left" as the merge method
merged_data = pd.merge(merged_data, df_market_caps, on="Ticker", how="left")
merged_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Sentiment_Score,New_Covid_Cases,MarketCap
0,2013-04-30,32.560001,33.110001,32.279999,33.099998,27.390682,75165200,MSFT,5.60,NaN,2396829515776
1,2013-05-01,32.930000,33.080002,32.599998,32.720001,27.076227,54330900,MSFT,0.04,NaN,2396829515776
2,2013-05-02,32.630001,33.169998,32.389999,33.160000,27.440329,46059500,MSFT,3.02,NaN,2396829515776
3,2013-05-03,33.230000,33.520000,33.080002,33.490002,27.713415,46784600,MSFT,2.66,NaN,2396829515776
4,2013-05-06,33.419998,33.910000,33.250000,33.750000,27.928564,40978300,MSFT,2.20,NaN,2396829515776


In [9]:
merged_data.to_csv("merged_data.csv", index=False)